In [1]:
#imports
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import time
import math
from collections import defaultdict
from numpy.linalg import inv

if 'oldsysstdout' not in locals():
    import sys
    oldsysstdout = sys.stdout
    class flushfile():
        def __init__(self, f):
            self.f = f
        def __getattr__(self,name): 
            return object.__getattribute__(self.f, name)
        def write(self, x):
            self.f.write(x)
            self.f.flush()
        def flush(self):
            self.f.flush()
    sys.stdout = flushfile(sys.stdout)

In [2]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    print imgpts
    return img
axis = np.float32([[50,50,-30], [40,0,0], [0,40,0]]).reshape(-1,3)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

def nothing(x):
    pass

In [12]:
visualize = False
def detect(img):
    img_gs = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_gs = clahe.apply(img_gs)
    img_gs = np.float32(img_gs)
    dst = cv2.cornerHarris(img_gs,2,3,0.04)
    #result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)
    
    coords = np.transpose(np.nonzero(dst>0.01*dst.max()))
    # Threshold for an optimal value, it may vary depending on the image.
    #if visualize:
        #img[dst>0.01*dst.max()]=[0,0,255]
    #print coords
    #cv2.imshow('dst',img)
    return (coords,img_gs)

def dist2(a, b):
    return (a[0]-b[0])*(a[0]-b[0]) + (a[1]-b[1])*(a[1]-b[1])

def subimage(image, center, ts, tc, width, height):
    
    v_x = (tc, ts)
    v_y = (-ts,tc)
    s_x = center[0] - v_x[0] * (width / 2) - v_y[0] * (height / 2)
    s_y = center[1] - v_x[1] * (width / 2) - v_y[1] * (height / 2)

    mapping = np.array([[v_x[0],v_y[0], s_x],
                        [v_x[1],v_y[1], s_y]])


    return cv2.warpAffine(
        image,
        mapping,
        (int(width), int(height)),
        flags=cv2.WARP_INVERSE_MAP,
        borderMode=cv2.BORDER_REPLICATE)

def subimage2(image, c1, c2, width):
    w = c1[0]-c2[0]
    h = c1[1]-c2[1]
    center = topoint((c1+c2)/2)
    c = math.sqrt(dist2(c1, c2))
    ts = -w/c
    tc = h/c
    #print (topoint(c1), topoint(c2), center, ts, tc, w, h, c)
    return subimage(image, center, ts, tc, width, c)

def circleCoords(coords, mask_zero, radius = 2, maxSize = 22):
    mask = mask_zero[:,:,0].copy()
    for c in coords:
        cv2.circle(mask, (c[1],c[0]), radius,255,-1)
    ret, markers, stats, centroids = cv2.connectedComponentsWithStats(mask)
    if visualize:#for visualization only
        max_ = np.amax(markers)
        markers *= 255/max_
        markers = markers.astype(np.uint8)
        markers_col = cv2.applyColorMap(markers, cv2.COLORMAP_JET)
    candidates = []
    for (i,c) in enumerate(centroids):
        if  (stats[i,cv2.CC_STAT_WIDTH] < maxSize and stats[i,cv2.CC_STAT_HEIGHT] < maxSize and
            (stats[i,cv2.CC_STAT_WIDTH] > 4*radius or stats[i,cv2.CC_STAT_HEIGHT] >4*radius)):
            if visualize:
                cv2.circle(markers_col, (int(c[0]),int(c[1])), maxSize/2,(0,0,255),1)
            candidates += [c]
    if visualize:#for visualization only
        cv2.imshow('markers_col', markers_col)
    return candidates
    

def topoint(x):
    return (int(x[0]),int(x[1]))
    

def findGraph(candidates, img_gs):
    mask = np.zeros_like(img_gs)
    graph = defaultdict(lambda:[])
    if visualize:
        cv2.imshow('clahe', img_gs)
    
    for (i,c) in enumerate(candidates):
        for j in range(i):
            sub = subimage2(img_gs, c, candidates[j],10)
            min_ = np.amin(sub, axis = 1)
            
            
            if min_.mean() < 40:
                if visualize:
                    cv2.line(mask, topoint(c), topoint(candidates[j]),255,1)
                graph[i] += [j]
                graph[j] += [i]
    return graph, mask

#http://stackoverflow.com/a/246063
def CrossProductZ(a,b):
    return a[0] * b[1] - a[1] * b[0]

def Orientation(a, b, c):
    return CrossProductZ(a, b) + CrossProductZ(b, c) + CrossProductZ(c, a)

def OrientationC(g, a, b, c):
    return Orientation(g[a],g[b],g[c])

def add1(a):
    return np.append(a,[1])

def findHouse(graph, mask_, img, candidates, primary=5, secondary=4, name='_X', color=(0,0,255),img_gs=None):
    if visualize:
        mask = mask_.copy()
    houses = []
    for i in graph:
    #looping
        if len(graph[i]) != 2:
            continue
        [a,b] = graph[i]
        if len(graph[a]) != primary or len(graph[a]) != primary:
            continue
        ok = 1
        for c in graph[a]:
            if c != b and c!= i and len(graph[c]) != secondary:
                ok = 0
                break
        if ok == 0:
            continue
        #print [a]+graph[a]
        if OrientationC(candidates,i,a,b)<0:
            (a,b) = (b,a)
        for x in [a]+graph[a]:
            for y in graph[x]:
                if x<y:
                    if visualize:
                        cv2.line(mask, topoint(candidates[x]), topoint(candidates[y]),255,3)
                    cv2.line(img, topoint(candidates[x]), topoint(candidates[y]),color,1)
        used = {i: 1, a:1, b:1}
        #print used
        x = 0
        s = 0
        if name == '_X':
            for j in filter(lambda x: x not in used, graph[a]):
                s_ = 0
                for k in graph[j]:
                    s_ += dist2(candidates[j],candidates[k])
                if x == 0 or s_ < s:
                    x = j
                    s = s_
        #else:
        #    return
            used[x] = 1
        [c,d] = filter(lambda x: x not in used, graph[a])
        if OrientationC(candidates,x,c,d)<0:
            (c,d) = (d,c)
    
        points_imdg = [candidates[__] for __ in [a,b,c,d]]#,x,i
        if name == '_X':
            points_img = [candidates[__] for __ in [a,b,c,d,x]]#,x,i
            new_3d = np.float32([[100,0,0],[0,0,0],[100,100,0],[0,100,0],[50,50,0]])
        else:
            points_img = [candidates[__] for __ in [a,b,c,d]]#,x,i
            new_3d = np.float32([[100,0,0],[0,0,0],[100,100,0],[0,100,0]])
        p_img = np.array(points_img, np.float32)
        #p_new = np.array(points_new, np.float32)
        #p_new3 = np.array(points_new_3, np.float32)
        cv2.cornerSubPix(img_gs,p_img,(11,11),(-1,-1),criteria)
        #cv2.circle(img, topoint(p_img[5]), 5, (255,0,0),-1)
        cv2.circle(img, topoint(p_img[0]), 5, (0,0,255),-1)#tr
        cv2.circle(img, topoint(p_img[1]), 5, (0,255,0),-1)#tl
        #cv2.circle(img, topoint(p_img[4]), 5, (255,255,255),-1)
        cv2.circle(img, topoint(p_img[2]), 5, (0,255,255),-1)#br
        cv2.circle(img, topoint(p_img[3]), 5, (255,0,255),-1)#bl
    #
        return draw3d(img,p_img,new_3d)
    #   
        objpoints.append(p_new)
        imgpoints.append(p_img)
        
        #return
        
        #retval,rvecs, tvecs, inliers = cv2.solvePnPRansac(p_new, p_img, mtx, dist)
        #print (retval,rvecs, tvecs, inliers)
        #imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        #print p_img
        #print p_new
        #print axis
        #print imgpts
        #print jac
        #retval, M, inliers = cv2.estimateAffine3D(p_new, p_img)
        #print np.dot([49,49,1],M)
        #print (retval, M, inliers)
        #return
        circ = cv2.imread('circle.png')
        global persp, persp_inv
        persp = cv2.getPerspectiveTransform(p_img, p_new)
        persp_inv = cv2.getPerspectiveTransform(p_new, p_img)
        print persp
        print persp_inv
        trans = cv2.warpPerspective(img, persp, (100, 100))
        trans_inv = cv2.warpPerspective(circ, persp_inv, (640, 480))
        cv2.imshow('trans',trans)
        cv2.imshow('trans_inv',trans_inv)
        homo = cv2.findHomography(p_img, p_new3)
        print homo
        return
        img_ = draw(img,p_img,imgpts)
        cv2.imshow('img_',img_)
        if OrientationC(candidates,i,a,b)<0:
            (a,b) = (b,a)
        houses += [(i,x,a,c,d,b)]
        return (points_img,points_new)
        if visualize:          
            cv2.imshow('mask'+name, mask)
    return (None,None)

#new_3d = np.float32([[100,0],[0,0],[100,100],[0,100],[50,50]])
goal_3d = np.float32([[50,50,-1]])
def draw3d(img, pts, new_3d):
    fx = 0.5 + cv2.getTrackbarPos('focal', 'dst') / 50.0
    h, w = img.shape[:2]
    K = np.float64([[fx*w, 0, 0.5*(w-1)],
                    [0, fx*w, 0.5*(h-1)],
                    [0.0,0.0,      1.0]])
    dist_coef = np.zeros(4)
    ret, rvec, tvec = cv2.solvePnP(new_3d, pts, K, dist_coef)
    #print ret
    #min_ = np.min(pts_3d, axis=0)
    #max_ = np.max(pts_3d, axis=0)
    #print min_
    #print max_
    #(x0, x1, y0, y1) = (min_[0], max_[0], min_[1], max_[1])
    #verts = goal_3d * [(x1-x0), (y1-y0), -(x1-x0)*0.3] + (x0, y0, 0)
    goal_3d[0,2] = -1 * cv2.getTrackbarPos('height', 'dst')
    verts = cv2.projectPoints(goal_3d, rvec, tvec, K, dist_coef)[0].reshape(-1, 2)
    cv2.circle(img, topoint(verts[0]), 5, (255,255,255),-1)
    for p in pts:
        cv2.line(img, topoint(p), topoint(verts[0]), (255,255,255),2, -1)
    #print pts_3d
    print verts

mypath = 'frames'
if 'cap' in locals():
    cap.release()
#cap = cv2.VideoCapture(0)
#while(True):
objpoints = []
imgpoints = []

cv2.namedWindow('dst')
cv2.createTrackbar('focal', 'dst', 25, 50, nothing)
cv2.createTrackbar('height', 'dst', 25, 50, nothing)
for f in listdir(mypath):
    if not isfile(join(mypath, f)):
        continue
    if f[-3:] != 'jpg':
        continue
    
    #ret, frame = cap.read()
    frame = cv2.imread(join(mypath, f))
    start = time.time()
    (coords_,img_gs) = detect(frame)
    candidates = circleCoords(coords_, np.zeros_like(frame))
    graph, mask = findGraph(candidates, img_gs)
    done = False
    while not done:
        findHouse(graph, mask, frame, candidates, 5, 4, '_X', img_gs=img_gs)
        cv2.imshow('dst',frame)
        key = cv2.waitKey(1000)
        if key & 0xFF == ord('q'):
            break
        if key & 0xFF == ord(' '):
            done = True
    if key & 0xFF == ord('q'):
        break
    #findHouse(graph, mask, frame, candidates, 3, 2, '_N', color=(0,255,0))
    #end = time.time()
    
    #print end - start
    #break
if 'cap' in locals():
    cap.release()
#coords_

#R(x - t)
#R
#t


[[ 395.7532959   199.52296448]]
[[ 395.7532959   199.52296448]]
[[ 395.7532959   199.52296448]]
[[ 395.7532959   199.52296448]]
[[ 395.7532959   199.52296448]]
[[ 395.7532959   199.52296448]]
[[ 392.18252563  210.74375916]]
[[ 392.18252563  210.74375916]]
[[ 392.16705322  210.7802124 ]]
[[ 392.16705322  210.7802124 ]]
[[ 392.11254883  210.86405945]]
[[ 392.00170898  210.40118408]]
[[ 393.18640137  209.94796753]]
[[ 401.88885498  176.57015991]]
[[ 406.97674561  199.43460083]]
[[ 407.97738647  199.77166748]]
[[ 407.93341064  200.17140198]]
[[ 408.1144104   201.14953613]]
[[ 408.70663452  204.70825195]]
[[ 407.93841553  208.64080811]]
[[ 419.4473877   209.98693848]]
[[ 419.4473877   209.98693848]]
[[ 461.69442749  223.19049072]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.85845947  270.47998047]]
[[ 471.8

In [55]:
np.dot(persp,np.array([50,50,2]))

array([-565.79678591, -168.18240705,    1.979676  ])

In [25]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (480,640), None, None)
ret, mtx, dist, rvecs, tvecs

(0.24704672475739936,
 array([[  1.00419761e+03,   0.00000000e+00,   2.48492695e+02],
        [  0.00000000e+00,   1.10274617e+03,   3.16436043e+02],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]]),
 array([[ -1.53603750e+00,   2.68088999e+01,  -3.92649562e-02,
           7.21702461e-02,  -2.39172823e+02]]),
 [array([[ 0.66253312],
         [ 0.23887962],
         [ 0.00985214]]), array([[ 0.66291993],
         [ 0.23908822],
         [ 0.01004852]]), array([[ 0.66330659],
         [ 0.23937298],
         [ 0.0105343 ]]), array([[ 0.65924246],
         [ 0.24195699],
         [ 0.01035407]]), array([[ 0.65802251],
         [ 0.23669018],
         [ 0.01014907]]), array([[ 0.49487153],
         [ 0.1847021 ],
         [-0.02319952]]), array([[-0.42042229],
         [ 0.04697471],
         [ 0.09685658]]), array([[-0.44326945],
         [ 0.05831716],
         [ 0.11079589]]), array([[-0.50679087],
         [ 0.05336211],
         [ 0.120367  ]]), array([[-0.54638336],
 

In [21]:
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
objp

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 2.,  0.,  0.],
       [ 3.,  0.,  0.],
       [ 4.,  0.,  0.],
       [ 5.,  0.,  0.],
       [ 6.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  1.,  0.],
       [ 2.,  1.,  0.],
       [ 3.,  1.,  0.],
       [ 4.,  1.,  0.],
       [ 5.,  1.,  0.],
       [ 6.,  1.,  0.],
       [ 0.,  2.,  0.],
       [ 1.,  2.,  0.],
       [ 2.,  2.,  0.],
       [ 3.,  2.,  0.],
       [ 4.,  2.,  0.],
       [ 5.,  2.,  0.],
       [ 6.,  2.,  0.],
       [ 0.,  3.,  0.],
       [ 1.,  3.,  0.],
       [ 2.,  3.,  0.],
       [ 3.,  3.,  0.],
       [ 4.,  3.,  0.],
       [ 5.,  3.,  0.],
       [ 6.,  3.,  0.],
       [ 0.,  4.,  0.],
       [ 1.,  4.,  0.],
       [ 2.,  4.,  0.],
       [ 3.,  4.,  0.],
       [ 4.,  4.,  0.],
       [ 5.,  4.,  0.],
       [ 6.,  4.,  0.],
       [ 0.,  5.,  0.],
       [ 1.,  5.,  0.],
       [ 2.,  5.,  0.],
       [ 3.,  5.,  0.],
       [ 4.,  5.,  0.],
       [ 5.,  5.,  0.],
       [ 6.,  5.

In [13]:
tl = [30,10,1]
tr = [90,10,1]
br = [90,90,1]




criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
print objp
return

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
img = cv2.imread('frames/frame_00.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

if ret == True:
    corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

    # Find the rotation and translation vectors.
    rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

    img = draw(img,corners2,imgpts)
    cv2.imshow('img',img)
    k = cv2.waitKey(0) & 0xff
    if k == 's':
        cv2.imwrite(fname[:6]+'.png', img)

cv2.destroyAllWindows()


dbg = np.zeros((100,100,3))
dbg[tl[1],tl[0]]=255
dbg[tr[1],tr[0]]=255
dbg[br[1],br[0]]=255
print math.sqrt(dist2(tl, tr))
print math.sqrt(dist2(tr, br))
tr_tl = np.cross(tr,tl)
tr_br = np.cross(tr,br)
print tr_tl
print tr_br
#(x1,y1) = (tr_tl[0]/tr_tl[2], tr_tl[1]/tr_tl[2])
#d1 = math.sqrt(dist2((0,0),(x1,y1))) / 10.0
#x1 /= d1
#y1 /= d1

#(x2,y2) = (tr_br[0]/tr_br[2], tr_br[1]/tr_br[2])
#d2 = math.sqrt(dist2((0,0),(x2,y2))) / 10.0
#x2 /= d2
#y2 /= d2

#print (x1,y1)
#print (x2,y2)

normal =  tr_tl * tr_br
print normal
#top = candidates[x] + normal[0:1]*

cv2.imshow('dbg', dbg)
cv2.waitKey(0)

[[ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 2.  0.  0.]
 [ 3.  0.  0.]
 [ 4.  0.  0.]
 [ 5.  0.  0.]
 [ 6.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  1.  0.]
 [ 2.  1.  0.]
 [ 3.  1.  0.]
 [ 4.  1.  0.]
 [ 5.  1.  0.]
 [ 6.  1.  0.]
 [ 0.  2.  0.]
 [ 1.  2.  0.]
 [ 2.  2.  0.]
 [ 3.  2.  0.]
 [ 4.  2.  0.]
 [ 5.  2.  0.]
 [ 6.  2.  0.]
 [ 0.  3.  0.]
 [ 1.  3.  0.]
 [ 2.  3.  0.]
 [ 3.  3.  0.]
 [ 4.  3.  0.]
 [ 5.  3.  0.]
 [ 6.  3.  0.]
 [ 0.  4.  0.]
 [ 1.  4.  0.]
 [ 2.  4.  0.]
 [ 3.  4.  0.]
 [ 4.  4.  0.]
 [ 5.  4.  0.]
 [ 6.  4.  0.]
 [ 0.  5.  0.]
 [ 1.  5.  0.]
 [ 2.  5.  0.]
 [ 3.  5.  0.]
 [ 4.  5.  0.]
 [ 5.  5.  0.]
 [ 6.  5.  0.]]


SyntaxError: 'return' outside function (<ipython-input-13-9b3f3495b32f>, line 17)

In [14]:
cv2.__version__

'3.1.0'

In [23]:


if 'cap' in locals():
    cap.release()
cap = cv2.VideoCapture(0)
i = 0
while(True):
    ret, frame = cap.read()
    #coords_ = detect(frame)
    #clusters = mergeCoords(coords_)
    cv2.imwrite('frame_%d.jpg'%i,frame)
    cv2.imshow('dst',frame)
    i+=1
    if cv2.waitKey(500) & 0xFF == ord('q'):
        break
    #break
cap.release()